In [ ]:
using TensorOperations, TensorKit
import TensorKit.×
using FLoops
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./models/Hubbard_Z2SU2.jl")
# include("./models/tJ_Z2.jl")
@show pkgversion(TensorKit)
using JLD2, HDF5
using Strided
Strided.enable_threads()

In [ ]:
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8.0
para[:μ] = 4.0
para[:pspace] = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
Hij = get_op_Hubbard("hij", para)
@show Hij

Hijp = permute(Hij, ((2, 1), (4, 3)))
@show Hijp
@assert Hij == Hijp
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./models/tJ_Z2.jl")
include("./simple_update_Fermionic/simple_update.jl")
include("./fast_full_update_Fermionic/fast_full_update.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")

ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)

Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, ["Δₛ", "Δₛdag"], para; site1=[1, 1], site2=[0, 2], get_op=get_op_tJ)

nothing

In [ ]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")


In [ ]:
using TensorKit
using Zygote
using ChainRulesCore
import TensorKit.×
include("./iPEPS_Fermionic/iPEPS.jl")

In [ ]:
function f(A::TensorMap, B::TensorMap)
    # _, R = leftorth(A, ((1,), (2,)))
    U, S, Vdag, ϵ = tsvd(A, ((1,), (2,)); trunc=notrunc(), alg=TensorKit.SVD())
    # @tensor Ap[a, b] := A[a, c] * A[c, b]
    return @tensor C = Vdag[a, c] * B[c, a]
end

# 可以忽略某些参数的导数
function g(A::TensorMap, B::TensorMap)
    B = ChainRulesCore.ignore_derivatives(B)   # 不计算B的导数！！！！
    # _, R = leftorth(A, ((1,), (2,)))
    U, S, Vdag, ϵ = tsvd(A, ((1,), (2,)); trunc=notrunc(), alg=TensorKit.SVD())
    # @tensor Ap[a, b] := A[a, c] * A[c, b]
    return @tensor C = Vdag[a, c] * B[c, a]
end

pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

A = TensorMap(randn, Float64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
Ms = [A, A, A, A]
ipeps = iPEPS(Ms, 2, 2)
nothing

In [ ]:
function iPEPScontract(ipeps1::iPEPS, ipeps2::iPEPS)
    A = ipeps1[1, 1]
    B = ipeps2[1, 1]
    return @tensor C = A[l, t, p, r, b] * B'[r, b, l, t, p]
end

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂×U₁]((0, 0) => 2, (1, 1) => 1)
# pspace = ℝ^3
A = TensorMap(randn, Float64, pspace, pspace)
B = TensorMap(randn, Float64, pspace, pspace)

# 内含 QR, SVD, 以及复杂矩阵乘的函数，无论 pspace 对称性如何，都可以直接求梯度
grd = gradient((a, b) -> f(a, b), A, B)
@show grd
y, back = pullback((a, b) -> f(a, b), A, B)  # pullback 可以返回函数值和带系数的梯度生成器
@show y
@assert back(1.0) == grd
@assert back(2.0) == 2.0 .* grd

grd2 = gradient((a, b) -> g(a, b), A, B)
@show grd2
@assert grd[1] == grd2[1]
nothing

In [ ]:
# grd = gradient((a, b) -> iPEPScontract(a, b), ipeps, ipeps)
grd = gradient(ipeps, ipeps) do a, b
    b = ChainRulesCore.ignore_derivatives(b)   # 不计算B的导数,在匿名函数里面声明可以不改动底层函数代码
    iPEPScontract(a, b)
end
@show typeof(grd)
@show typeof(grd[1].Ms[1])
@show grd[1].Lx
@show grd[2]
nothing

In [ ]:
a = Matrix{Array}(undef, 2, 2)
a[1, 1] = [[3, 4, 5], [1, 1, 1]]
a[2, 2] = [1, 2, 3]
a[2, 1] = [1, 2, 3]
a[1, 2] = [1, 2, 3]

_, S, _ = tsvd(A, ((1,), (2,)); trunc=notrunc(), alg=TensorKit.SVD())
a[2, 1] = [diag(convert(Array, S)), diag(convert(Array, S))]
a[2, 2] = map(x -> diag(convert(Array, x)), [S, S])
@show a[2, 1]
@show a[2, 2]
@assert a[2, 1] == a[2, 2]

b = [1 4; 2 3]
@assert b isa Matrix
@show b
@show maximum(b)
nothing